In [11]:
#| default_exp components

In [2]:
#| export
from fasthtml.common import *
from monsterui.all import *
from datetime import datetime

In [25]:
#| export
def Layout(*children, title="My Blog", wide=False):
    """Main layout component with floating navigation and footer"""
    content_cls = "max-w-5xl mx-auto" if wide else "max-w-3xl mx-auto"
    return (
        Title(title),
        # Floating navigation bar
        Nav(
            Div(
                A("Ivan Dolgushev", href="/",
                  cls="text-lg font-bold text-gray-900 hover:text-blue-600 transition-colors"),
                Div(
                    A("Home", href="/", cls="text-sm font-medium text-gray-600 hover:text-blue-600 transition-colors"),
                    A("About", href="/about", cls="text-sm font-medium text-gray-600 hover:text-blue-600 transition-colors"),
                    A("Contact", href="/contact", cls="text-sm font-medium text-gray-600 hover:text-blue-600 transition-colors"),
                    cls="flex gap-6 items-center"
                ),
                cls="flex justify-between items-center w-full max-w-2xl px-6"
            ),
            cls=("fixed top-6 left-1/2 -translate-x-1/2 z-50 "
                 "bg-white/80 backdrop-blur-md border border-gray-200/50 "
                 "rounded-full shadow-lg py-6 px-4 w-[90%] max-w-3xl "
                 "transition-all duration-300 hover:shadow-xl hover:-translate-y-0.5")
        ),

        # Main content area
        Container(
            Div(children, cls=content_cls),
            # Footer
            Footer(
                P(f"© {datetime.now().year} Ivan Dolgushev. Built with FastHTML & MonsterUI.",
                  cls="text-center text-sm text-gray-500"),
                cls="py-12 mt-20 border-t border-gray-100"
            ),
            cls="min-h-screen px-4 pt-32 pb-12 bg-gray-50/50"
        )
    )

In [23]:
#| export
def PostCard(post: dict):
    """Clickable post preview card"""
    date_str = post['date'].strftime('%d %b %Y')
    return A(
        Div(
            H3(post['title'], 
               cls="text-xl font-medium text-gray-900 group-hover:text-blue-600 transition-colors"),
            Span(date_str, cls="text-sm text-gray-500 ml-auto"),
            cls="flex items-baseline justify-between gap-4"
        ),
        P(post['excerpt'], cls="text-gray-600 mt-2 text-sm leading-relaxed"),
        Div(
            *[Span(f"#{tag}", cls="text-xs font-semibold text-blue-600 bg-blue-50 px-2 py-1 rounded-full") 
              for tag in post.get('tags', [])],
            cls="mt-3 flex flex-wrap gap-2"
        ),
        href=f"/post/{post['slug']}",
        cls="block py-6 border-b border-gray-200 hover:border-blue-300 hover:bg-gray-50/50 -mx-4 px-4 rounded-lg transition-all group"
    )

In [19]:
#| export
def FilterSection(all_categories: list[str], selected_categories: list[str], enabled_categories: list[str]):
    """Category filter chips with multi-select support"""
    
    def build_url(cats):
        return f"/?{'&'.join([f'category={c}' for c in cats])}" if cats else "/"

    def chip(cat, label=None):
        """Render category filter chip"""
        is_all = cat is None
        is_selected = (not selected_categories) if is_all else (cat in selected_categories)
        is_enabled = is_all or cat in enabled_categories
        
        # calculate next state
        if is_all: next_cats = []
        elif is_selected: next_cats = [c for c in selected_categories if c != cat]
        else: next_cats = selected_categories + [cat]
  
        cls = ("bg-gray-900 text-white hover:bg-gray-800" if is_selected else 
                   "bg-gray-100 text-gray-600 hover:bg-gray-200" if is_enabled else 
                   "bg-gray-50 text-gray-300 cursor-not-allowed")
                   
        return A(label or cat.title(), 
                href=build_url(next_cats) if is_enabled else "#",
                cls=f"px-4 py-2 rounded-full text-sm font-medium transition-all duration-300 {cls}")

    return Div(
        chip(None, "All"),
        *[chip(cat) for cat in all_categories],
        cls="flex flex-wrap gap-2 justify-center mb-12"
    )

In [13]:
#| export
def PostContent(post: dict):
    """Full post content component for individual post pages"""
    date_str = post['date'].strftime('%B %d, %Y') if isinstance(post['date'], datetime) else str(post['date'])

    return Article(
        Header(
            H1(post['title'], cls="text-5xl font-bold mb-4 text-gray-900"),
            P(date_str, cls="text-gray-500 mb-8"),
            cls="mb-8"
        ),

        # Post content with prose styling
        NotStr(render_md(post['content'])),
        Footer(
            A("← Back to Home", href="/",
              cls="inline-block px-4 py-2 border-2 border-blue-600 text-blue-600 rounded-lg hover:bg-blue-600 hover:text-white transition-colors mt-12")
        ),

        cls="py-8 bg-white shadow-lg p-8 rounded-lg"
    )

In [14]:
#| export
def AboutContent():
    """About page content"""
    return Div(
        H1("About Me", cls="text-4xl font-bold mb-6"),
        Div(
            P("Welcome to my blog! This is where I share my thoughts and experiences.", cls="mb-4 text-lg"),
            P("Built with FastHTML and MonsterUI for a modern, responsive experience.", cls="mb-4 text-lg"),
            P("I'm passionate about technology, programming, and sharing knowledge with the community.", cls="text-lg"),
            cls="prose prose-lg"
        )
    )

In [15]:
#| export
def ContactForm():
    """Contact form component"""
    return Div(
        H1("Contact Me", cls="text-4xl font-bold mb-6"),
        Form(
            LabelInput("Name", id="name", placeholder="Your name"),
            LabelInput("Email", id="email", type="email", placeholder="your@email.com"),
            LabelTextArea("Message", id="message", rows=5, placeholder="What's on your mind?"),
            Button("Send Message", cls=ButtonT.primary),
            action="/contact/submit", method="post",
            cls="space-y-4 max-w-lg bg-white shadow-lg p-8 rounded-lg"
        )
    )

In [16]:
#| export
def NotFoundContent():
    """404 page content"""
    return Div(
        H1("404 - Page Not Found", cls="text-4xl font-bold text-center mt-20"),
        P("The page you're looking for doesn't exist.", cls="text-center mt-4 text-lg text-gray-600"),
        Div(
            A("Go Home", href="/",
              cls="inline-block px-6 py-3 bg-blue-600 text-white rounded-lg hover:bg-blue-700 transition-colors mt-6"),
            cls="text-center"
        )
    )

In [26]:
from nbdev.export import nb_export
nb_export('01_components.ipynb','../')